In [1]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli fuzzywuzzy
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 15.5 MB/s eta 0:00:0000:0100:01
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)

[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


/Users/alex/Desktop/Home/fpl-challenge-optimisations/venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
solve_season = '2024-25'
solve_gameweek = 32
load_projections_from_file = False

In [4]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections

if not load_projections_from_file:
    point_projections = generate_projections()
    stat_projections = generate_stat_projections()

    projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)
    print('Generated new projections.')
else:
    try:
        projections_data = pd.read_csv('gameweek_projections.csv')
        print('Loaded projections from CSV file.')
    except FileNotFoundError:
        print('Tried to load from CSV file, but it does not exist. Generating new projections...')
        point_projections = generate_projections()
        stat_projections = generate_stat_projections()
        projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)

Generated new projections.


### Player Manipulation

### Player Force/Banning

In [5]:
def fuzzy_ban_players(df, ban_ids):
    while True:
        search_name = input("Enter player name to ban (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to ban (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    ban_ids.append(selected_index)
                    print(f"Banned: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return ban_ids

ban_ids = []
ban_ids = fuzzy_ban_players(projections_data, ban_ids)
print("Final ban list (indices):", ban_ids)

Final ban list (indices): []


In [6]:
def fuzzy_force_players(df, force_ids):
    while True:
        search_name = input("Enter player name to force (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to force (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    force_ids.append(selected_index)  # Add to force_ids instead of ban_ids
                    print(f"Forced: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return force_ids

force_ids = []
force_ids = fuzzy_force_players(projections_data, force_ids)
print("Final force list (indices):", force_ids)

Final force list (indices): []


# 2024/25 GW32 Challenge: All-Rounder - Must use the 2-2-2 formation

### Optimisation

In [7]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 7
model += plp.lpSum(lineup) == 7

# List players by index to be EXCLUDED from the lineup
for id in ban_ids:
    model += lineup[id] == 0

for id in force_ids:
    model += lineup[id] == 1

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeepers
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# Exactly 2 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) == 2

# Exactly 2 Midfielders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) == 2

# Exactly 2 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) == 2

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

def solution_to_txt(players_dict, filename="solution.txt", encoding="utf-8"):
    total_points = 0
    total_cost = 0
    with open(filename, 'w', encoding=encoding) as f:
        f.write(f'Current Date & Time: {time.strftime("%Y-%m-%d")} - {time.strftime("%H:%M:%S")}\n')
        for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
            if position in players_dict:
                f.write(f"\n{position}:\n")
                for player in players_dict[position]:
                    captain_str = " (C)" if player['Captain'] else ""
                    points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                    f.write(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}\n")
                    total_points += points
                    total_cost += player['Cost']
        f.write(f"\nTotal Predicted Points: {round(total_points, 2)}\n")
        f.write(f"Total Cost: {round(total_cost, 2)}m\n")
        
solution_to_txt(selected_players, "optimal_solution.txt")
print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alex/Desktop/Home/fpl-challenge-optimisations/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/91/3jnk9jzn3kx1mwsfzhddwbww0000gn/T/ee47e4a886834c058487fe8b98d767cf-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/91/3jnk9jzn3kx1mwsfzhddwbww0000gn/T/ee47e4a886834c058487fe8b98d767cf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 499 COLUMNS
At line 5516 RHS
At line 6011 BOUNDS
At line 6988 ENDATA
Problem MODEL has 494 rows, 976 columns and 2440 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 77.6 - 0.00 seconds
Cgl0004I processed model has 493 rows, 975 columns (975 integer (975 of which binary)) and 2378 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 0 integers unsatisfied sum 

In [8]:
def print_top_scorers_by_position(projections_data):
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        top_scorers = projections_data[projections_data['Position'] == position].nlargest(10, 'Predicted_Points')
        print(f'Top {position}s: ')
        display(top_scorers)

if print_solution:
    print_top_scorers_by_position(projections_data)

projections_data.to_csv('gameweek_projections.csv', index=False)